# Accessing USGS data

### This notebook does the following:  
* Load a single USGS point
* Look at the metadata
* Get multiple radiation datasets
* Calculate albedo from the datasets
* Plot the albedo evolution

In [ ]:
# imports
from datetime import datetime
import pandas as pd
import numpy as np

from metloom.pointdata import USGSPointData

# For rendering in readthedocs
import plotly.offline as py
py.init_notebook_mode(connected=True)

In [ ]:
# Define a point known to have solar measurements
pt = USGSPointData("395709105582701", "Blue Ridge Meteorological Station NR Fraser")

# start data and end date
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 7, 1)
# Define a list of variables we want
incoming_sw = pt.ALLOWED_VARIABLES.DOWNSHORTWAVE
outgoing_sw = pt.ALLOWED_VARIABLES.UPSHORTWAVE
depth = pt.ALLOWED_VARIABLES.SNOWDEPTH
variables = [incoming_sw, outgoing_sw, depth]



In [ ]:
# LETS GET THAT DATA
df = pt.get_hourly_data(start_date, end_date, variables)
df.head(10)

In [ ]:
# Check out this data!
df = df.reset_index().set_index("datetime")
var_names = [v.name for v in variables]

# Sample to just the vars
df_rad = df.loc[:, var_names]

df_rad.loc[:, [incoming_sw.name, outgoing_sw.name]].plot()



In [ ]:
# Let's get the data in a more usable state
sw_thresh = 10

# mask the SW to decent values
df_rad[incoming_sw.name] = df_rad[incoming_sw.name].mask(df_rad[incoming_sw.name] < sw_thresh, np.nan)
df_rad[outgoing_sw.name] = df_rad[outgoing_sw.name].mask(df_rad[outgoing_sw.name] < sw_thresh, np.nan)

# Resample to daily, based on mean values
df_rad = df_rad.resample("D").mean()

# Plot again
df_rad.loc[:, [incoming_sw.name, outgoing_sw.name]].plot()


### Now we can think Albedo

The daily data plot looks much better. Now we can start to think about albedo.
We can use the SW to calcaulate albedo next

In [ ]:
# Calculate albedo
albedo_var = "ALBEDO"
df_rad[albedo_var] = df_rad[outgoing_sw.name] / df_rad[incoming_sw.name]
df_rad[albedo_var] = df_rad[albedo_var].mask(df_rad[albedo_var] < 0.1, np.nan)
df_rad[albedo_var] = df_rad[albedo_var].mask(df_rad[albedo_var] > 1, np.nan)
df_rad[albedo_var].plot()

### Better plotting
Awesome - now we have albedo. We haven't used snowdepth yet,
so let's create a better plot that shows how the two relate

In [ ]:
# Get plotly for a nicer plot
# !pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_rad.index, y=df_rad[albedo_var], name=albedo_var),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        x=df_rad.index, y=df_rad[depth.name].diff(), name=f"{depth.name} signal", mode="lines+markers"
    ), secondary_y=True, 
)

fig.add_trace(
    go.Scatter(
        x=df_rad.index, y=df_rad[depth.name], name=f"{depth.name}", opacity=0.3,
    ), secondary_y=True, 
)

fig.update_layout(
    # template='plotly_dark',
    title=f'{pt.name}',
    xaxis=dict(title='Date'),
    yaxis=dict(
        title=f'Unitless',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        tickvals=[.25, .5, .75, 1.0],
        range=[.25, 1]
    ),
    yaxis2=dict(
        title=f'[m]',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'), overlaying='y', side='right'
    )
)

# Show the plot
fig.show()

### Summary
* We started with a point that we knew had shortwave measurements.
* Next, we pulled all the necessary data, cleaned it, and resampled to daily.
* We calculated albedo, and plotted it to see how the snow